In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
from scipy import sparse
import sys
sys.path.append("../src")
from driedger import *
from particle import *
from bayes import *
from probutils import *
from audioutils import *
import time

In [ ]:
sr = 44100
win = 2048
hop = win//2
min_freq = 0
max_freq = 8000
kmin = int(min_freq*win/sr)
kmax = int(max_freq*win/sr)+1

p = 5
c = 3 # Diagonal
r = 3 # Repeated activation
stereo = True

#ytarget = load_corpus("../target/Beatles_LetItBe.mp3", sr=sr, stereo=stereo)
#ycorpus = load_corpus("../corpus/Bees_Buzzing.mp3", sr=sr, stereo=stereo)

#ytarget = load_corpus("../target/fma_small/000/000141.mp3", sr=sr, stereo=stereo)
#ytarget = load_corpus("../target/Snakecharm-2-fullmixloop.wav", sr=sr, stereo=stereo)
#ycorpus = load_corpus("../corpus/EdenVIP2", sr=sr, stereo=stereo)

ytarget = load_corpus("../target/ParadisoBlip2.wav", sr=sr, stereo=stereo)
ycorpus = load_corpus("../corpus/BigOCorpus", sr=sr, stereo=stereo)

#ytarget = load_corpus("../target/aha.mp3", sr=sr, stereo=stereo)
#ycorpus = load_corpus("../corpus/theoshift.mp3", sr, True)

ipd.Audio(ytarget, rate=sr)

## Driedger Version

In [ ]:
W1L, _ = get_windowed(ytarget[0, :], win)
W1R, _ = get_windowed(ytarget[1, :], win)
VL = np.abs(np.fft.fft(W1L, axis=0)[kmin:kmax+1, :])
VR = np.abs(np.fft.fft(W1R, axis=0)[kmin:kmax+1, :])
V = np.concatenate((VL, VR), axis=0)

WSoundL, _ = get_windowed(ycorpus[0, :], win)
WSoundR, _ = get_windowed(ycorpus[1, :], win)
WL = np.abs(np.fft.fft(WSoundL, axis=0)[kmin:kmax+1, :])
WR = np.abs(np.fft.fft(WSoundR, axis=0)[kmin:kmax+1, :])
W = np.concatenate((WL, WR), axis=0)

tic = time.time()
HGT = get_musaic_activations(V, W, L=50, r=r, p=p, c=c, verbose=False)
#HGT = get_basic_KL_activations(V, W, L=50, verbose=True)
print("Elapsed time Driedger: {:.3f}".format(time.time()-tic))
WHGT = W.dot(HGT)
driedger_fit = np.sum(V*np.log(V/WHGT) - V + WHGT)

In [ ]:
import torch
HGT = np.array(HGT, dtype=np.float32)
yL = do_windowed_sum(torch.from_numpy(WSoundL), torch.from_numpy(HGT), win, hop)
yR = do_windowed_sum(torch.from_numpy(WSoundR), torch.from_numpy(HGT), win, hop)

y = np.array([yL.cpu().numpy(), yR.cpu().numpy()]).T
y = y/np.max(y)

ipd.Audio(y.T, rate=sr)

## Our Version

In [ ]:
pparticle = p
pd = 0.95
temperature = 10
L = 10
P = 2000
gamma = 0
neff_thresh = 0.1*P
tic = time.time()


feature_params = dict(
    win=win,
    sr=sr,
    min_freq=min_freq,
    max_freq=max_freq,
    use_stft=True,
    use_mel=False,
    mel_bands=40,
    use_zcs=False,
)
particle_params = dict(
    p=p,
    pfinal=p,
    pd=pd,
    temperature=temperature,
    L=L,
    P=P,
    gamma=gamma,
    r=r,
    neff_thresh=neff_thresh,
    proposal_k=0,
    use_top_particle=False,
    alpha=0.1
)

use_mic = False
pf = ParticleFilter(ycorpus, feature_params, particle_params, 'cuda', use_mic)
if not use_mic:
    %snakeviz ygen = pf.process_audio_offline(ytarget)
print("Elapsed time particle filter: {:.3f}".format(time.time()-tic))

In [ ]:
ipd.Audio(ygen.T, rate=sr)

In [ ]:
times = 1000*np.array(pf.frame_times)
cutoff = 1000*hop/sr
plt.plot(times)
plt.plot(cutoff*np.ones_like(times))
plt.xlabel("Frame Index")
plt.ylabel("Processing Time (ms)")

## Examine Results

In [ ]:
H = pf.get_H()

prefix = "../results/Particle_p{}_pd{}_temperature{}_L{}_r{}_P{}_neff{}".format(p, pd, temperature, L, r, P, neff_thresh)

wsmax, neff = pf.wsmax, pf.neff

a1 = get_activations_diff(HGT, p)
a2 = get_activations_diff(H, p)
r1 = get_repeated_activation_itervals(HGT, p)
r2 = get_repeated_activation_itervals(H, p)

"""
WH = W.dot(H)
fit = np.sum(V*np.log(V/WH) - V + WH)
H = np.array(H, dtype=np.float32)
WSoundL = np.array(WSoundL, dtype=np.float32)
WSoundR = np.array(WSoundR, dtype=np.float32)
yL = do_windowed_sum(torch.from_numpy(WSoundL), torch.from_numpy(H), win, hop)
yR = do_windowed_sum(torch.from_numpy(WSoundR), torch.from_numpy(H), win, hop)
y = np.array([yL.cpu().numpy(), yR.cpu().numpy()]).T
y = y/np.max(y)

y = y/np.max(y)
wavfile.write("{}.wav".format(prefix), sr, y)
"""

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.plot(a1)
plt.plot(a2)
plt.legend(["Driedger: Mean {:.3f}".format(np.mean(a1)),
            "Particle Filter: Mean {:.3f}".format(np.mean(a2))])
plt.title("Activation Changes over Time, Temperature {}".format(temperature))
plt.xlabel("Timestep")
plt.subplot(232)
plt.hist(a1, bins=np.arange(p+2), alpha=0.5)
plt.hist(a2, bins=np.arange(p+2), alpha=0.5)
plt.title("Activation Changes Histogram")
plt.xlabel("Number of Activations Changed")
plt.ylabel("Counts")
plt.legend(["Driedger", "Particle Filter"])
plt.subplot(233)
plt.hist(r1, bins=np.arange(c*20), alpha=0.5)
plt.hist(r2, bins=np.arange(c*20), alpha=0.5)
plt.title("Repeated Activations Histogram, r={}".format(r))
plt.xlabel("Repeated Activation Distance")
plt.ylabel("Counts")
plt.legend(["Driedger", "Particle Filter"])
plt.subplot(234)
plt.plot(wsmax)
plt.title("Max Probability, Overall Fit: {:.3f} ({:.3f}x)\nDriedger Fit: {:.3f}".format(pf.fit, pf.fit/driedger_fit, driedger_fit))
plt.xlabel("Timestep")
plt.subplot(235)
plt.plot(neff)
plt.xlabel("Timestep")
plt.title("Number of Effective Particles (Median {:.2f})".format(np.median(neff)))
plt.subplot(236)
diagsgt = get_diag_lengths(HGT, p)
diags = get_diag_lengths(H, p)
plt.hist(diagsgt, bins=np.arange(30), alpha=0.5)
plt.hist(diags, bins=np.arange(30), alpha=0.5)
plt.legend(["Driedger Mean: {:.3f} (c={})".format(np.mean(diagsgt), c),
            "Particle Filter Mean: {:.3f} ($p_d$={})".format(np.mean(diags), pd)])
plt.xlabel("Diagonal Length")
plt.ylabel("Counts")
plt.title("Diagonal Lengths")
plt.tight_layout()
plt.savefig("{}.svg".format(prefix), bbox_inches='tight')

ipd.Audio(y.T, rate=sr)

In [ ]:
plt.figure(figsize=(8, 2.5))
bins=np.linspace(0, 650, 30)
plt.hist(1000*diagsgt*hop/sr, alpha=0.5, bins=bins)
plt.hist(1000*diags*hop/sr, alpha=0.5, bins=bins)
plt.legend(["Driedger Mean (c={}): {}".format(c, int(np.mean(diagsgt)*1000*hop/sr)),
            "Particle Filter Mean ($p_d$={}): {}".format(pd, int(np.mean(diags)*1000*hop/sr))])
plt.xlabel("Activation Length (Milliseconds)")
plt.ylabel("Counts")
plt.title("Activation Lengths Distributions on Big-O Corpus")
plt.savefig("ActivationsDist.svg", bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(311)
plt.imshow(V, cmap='magma_r', aspect='auto')
plt.title("V")
plt.colorbar()
plt.subplot(312)
plt.imshow(WHGT, cmap='magma_r', aspect='auto')
plt.title("Driedger")
plt.colorbar()
plt.subplot(313)
plt.imshow(WH, cmap='magma_r', aspect='auto')
plt.title("My Technique")
plt.colorbar()